In [116]:
import ipywidgets as widgets
import ipympl
from IPython.display import display
from localpackage.game import game
from localpackage.utils import returnFreq

from bqplot import *
import bqplot.pyplot as plt

import numpy as np
import math
regions=['UK','EW','EN','SC','WA','NI','GB']
lbeq=widgets.Label(value='=',layout=widgets.Layout(display="flex", justify_content="flex-start", width="10px"))
ClaimantOut = widgets.Output(layout=widgets.Layout(display="flex", justify_content="flex-start", width="420px", border='solid'))
DependentOut= widgets.Output(layout=widgets.Layout(display="flex", justify_content="flex-start", width="420px", border='solid')) 

In [36]:
#Game
dr=widgets.FloatSlider(value=-0.5,min=-3, max=3, description='Disc. Rate:')
td=widgets.DatePicker(description='Trial Date:',disabled=False)
og=widgets.Dropdown(
    options=['Ogden 7', 'Ogden 8'],
    value='Ogden 8',
    description='Ogden:',
    disabled=False,
)
fatal=widgets.Checkbox(value=False,description='Fatal',disabled=False,indent=False,layout=widgets.Layout(width='100px'))
yrattained=widgets.Checkbox(value=False,description='Auto Year Attained',disabled=False,indent=False,layout=widgets.Layout(width='200px'))
i1=[dr,td,og,fatal,yrattained]
h1=widgets.HBox(i1)
display(h1)
g={'discountRate':dr.value,'Ogden':7}

In [37]:
#Claimant
label=widgets.Label(value='Claimant:',layout=widgets.Layout(width='100px'))
age=widgets.FloatSlider(value=30,min=0,max=125, step=1,description='Age at trial:', layout=widgets.Layout(width='400px'))
sex=widgets.Dropdown(
    options=['Male', 'Female'],
    value='Male',
    description='Sex:',
    disabled=False,
    layout=widgets.Layout(width='175px')
)
rg=widgets.Dropdown(
    options=regions,
    value='UK',
    description='Region:',
    disabled=False,
    layout=widgets.Layout(width='175px'),
)
deltale=widgets.FloatText(value=0.0, description='deltaLE:', disabled=False, layout=widgets.Layout(width='150px'))
ageatdeath=widgets.FloatSlider(value=age.value-5,min=0,max=age.value, step=1,description='Age at death:', layout=widgets.Layout(width='400px'))

i2=[label,age,sex,rg,deltale,ageatdeath]
h2=widgets.HBox(i2)
display(h2)

In [38]:
#Dependent
labeld=widgets.Label(value='Dependent:',layout=widgets.Layout(width='100px'))
aged=widgets.FloatSlider(value=20,min=0,max=125,step=1, description='Age at trial:',layout=widgets.Layout(width='400px'))
sexd=widgets.Dropdown(
    options=['Male', 'Female'],
    value='Male',
    description='Sex:',
    disabled=False,
    layout=widgets.Layout(width='175px')
)
rgd=widgets.Dropdown(
    options=regions,
    value='UK',
    description='Region:',
    disabled=False,
    layout=widgets.Layout(width='175px')
)
deltaled=widgets.FloatText(value=0.0, description='deltaLE:', disabled=False,layout=widgets.Layout(width='150px')
)
i3=[labeld,aged,sexd,rgd,deltaled]
h3=widgets.HBox(i3)
display(h3)

In [39]:
#Row Claimant
lab=widgets.Label(value='Claimant:',layout=widgets.Layout(width='100px'))
r=widgets.FloatRangeSlider(
    value=[age.value, 125],
    min=0,
    max=125,
    step=1,
    description='Age Range:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=widgets.Layout(width='400px')
)
fr=widgets.Text(value='Y', description='Freq:',layout=widgets.Layout(width='175px'))
op=widgets.Text(value='AMI', description='Options:',layout=widgets.Layout(width='150px'))
i4=[lab,r,fr,op,lbeq,ClaimantOut]
h4=widgets.HBox(i4)
display(h4)

In [40]:
#Row Dependent
labd=widgets.Label(value='Dependent:',layout=widgets.Layout(width='100px'))
rd=widgets.FloatRangeSlider(
    value=[aged.value, 125],
    min=0,
    max=125,
    step=1,
    description='Age Range:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=widgets.Layout(width='400px')
)
frd=widgets.Text(value='Y', description='Freq:',layout=widgets.Layout(width='175px'))
opd=widgets.Text(value='AMID', description='Options:',layout=widgets.Layout(width='150px'))
i5=[labd,rd,frd,opd,lbeq,DependentOut]
h5=widgets.HBox(i5)
display(h5)

In [118]:



def getdataSet(og,region):
    if og=='Ogden 7':
        return {'year':2008,'region':region,'yrAttainedIn':2011}
    elif og=='Ogden 8':
        return {'year':2018,'region':region,'yrAttainedIn':2022}
    else:
        #error
        print('Wrong Ogden')

def getGame():
    claimants=[]
    if fatal.value:
        claimant = {'name': 'CLAIMANT', 'age': age.value,'aad':ageatdeath.value, 'sex': sex.value, 'dataSet': getdataSet(og.value,rg.value), 'deltaLE': deltale.value, 'cont':1}        
    else:
        claimant = {'name': 'CLAIMANT', 'age': age.value, 'sex': sex.value, 'dataSet': getdataSet(og.value,rg.value), 'deltaLE': deltale.value, 'cont':1}
    dependent= {'name': 'DEPENDENT', 'age': aged.value, 'sex': sexd.value, 'dependenton':'CLAIMANT', 'dataSet':getdataSet(og.value,rgd.value),'deltaLE': deltaled.value, 'cont':1}
    claimants.append(claimant)
    claimants.append(dependent)
    eg={'game':{'autoYrAttained':yrattained.value,'discountRate':dr.value/100, 'Ogden':7, 'claimants':claimants}}
    return game(eg)

def stringifyResult(result):
    return "Past ("+result[0]+") + Int("+result[1]+") + Future("+result[2]+") = " + result[3]

def getClaimantResult(thisGame):
    if (r.value[0]==r.value[1]):
        result=thisGame.getClaimant('CLAIMANT').M(r.value[0], freq=fr.value, options=op.value)
    else:
        result=thisGame.getClaimant('CLAIMANT').M(r.value[0], r.value[1], freq=fr.value, options=op.value)
    return result

def getDependentResult(thisGame):
    if (rd.value[0]==rd.value[1]):
        result=thisGame.getClaimant('DEPENDENT').M(rd.value[0], freq=frd.value, options=opd.value)
    else:
        result=thisGame.getClaimant('DEPENDENT').M(rd.value[0], rd.value[1], freq=frd.value, options=opd.value)
    return result
    

def updateBQplotC(thisGame,cRes):
    op.value=op.value.upper()
    curve=thisGame.getClaimant('CLAIMANT').getCurve()
    if not r.value[0]==r.value[1]:
        title=curve.getTitle(result=cRes,fromAge=r.value[0],toAge=r.value[1],freq=fr.value,cont=1,options=op.value)        
    else:
        title=curve.getTitle(result=cRes,fromAge=r.value[0],freq=fr.value,cont=1,options=op.value)

    #The plot
    LxNoI,Lx,Rng=curve.getCurve(options=op.value,cont=1)
    pdf_figC.title=title
    #scale
    if fatal.value:
        leftX=min(age.value,r.value[0],ageatdeath.value)-1
    else:
        leftX=min(age.value,r.value[0])-1
        
    pdf_lineC.scales['x'].min=leftX
    pdf_lineC.scales['x'].max=125
    pdf_lineC.scales['y'].min=0
    pdf_lineC.scales['y'].max=2
    pdf_lineC.x=Rng
    pdf_lineC.y=Lx
    #age at trial
    pdf_ageC.x=[age.value, age.value]
    pdf_ageC.y=[0, 1]
    #Age at death
    pdf_aad.x=[ageatdeath.value,ageatdeath.value]
    pdf_aad.y=[0,1]


    #Sort out fill range
    mask=(Rng>=r.value[0]) & (Rng<=r.value[1])
    R=Rng[mask]
    L=Lx[mask]
    #add interp points
    Ll=np.interp(r.value[0],Rng,Lx)
    Ul=np.interp(r.value[1],Rng,Lx)
    R=np.insert(R,0,r.value[0])
    L=np.insert(L,0,Ll)
    R=np.append(R,r.value[1])
    L=np.append(L,Ul)
    
    #single point
    y1=np.interp(r.value[0],Rng,Lx)
    pdf_startC.x=[r.value[0],r.value[0]]
    pdf_startC.y=[0,y1]
    y2=np.interp(r.value[1],Rng,Lx)
    pdf_endC.x=[r.value[1],r.value[1]]
    pdf_endC.y=[0,y2]

    

    pdf_fillC.x=R
    pdf_fillC.y=L

def updateBQplotD(thisGame,dRes):
    opd.value=opd.value.upper()
    curve=thisGame.getClaimant('DEPENDENT').getCurve()
    if not rd.value[0]==rd.value[1]:
        title=curve.getTitle(result=dRes,fromAge=rd.value[0],toAge=rd.value[1],freq=frd.value,cont=1,options=opd.value)
    else:
        title=curve.getTitle(result=dRes,fromAge=rd.value[0],freq=frd.value,cont=1,options=opd.value)

    #The plot
    LxNoI,Lx,Rng=curve.getCurve(options=opd.value,cont=1)
    pdf_figD.title=title
    #scale
    agegap=aged.value-age.value
    if fatal.value:
        leftX=min(age.value,r.value[0],ageatdeath.value)-1+agegap
    else:
        leftX=min(age.value,r.value[0])-1+agegap

        
    pdf_lineD.scales['x'].min=leftX
    pdf_lineD.scales['x'].max=125+agegap
    pdf_lineD.scales['y'].min=0
    pdf_lineD.scales['y'].max=2
    pdf_lineD.x=Rng
    pdf_lineD.y=Lx
    #Age
    pdf_ageD.x=[aged.value, aged.value]
    pdf_ageD.y=[0, 1]
    #Age at death
    pdf_aadd.x=[ageatdeath.value+agegap,ageatdeath.value+agegap]
    pdf_aadd.y=[0,1]


    #Sort out fill range
    mask=(Rng>=rd.value[0]) & (Rng<=rd.value[1])
    R=Rng[mask]
    L=Lx[mask]
    #add interp points
    Ll=np.interp(rd.value[0],Rng,Lx)
    Ul=np.interp(rd.value[1],Rng,Lx)
    R=np.insert(R,0,rd.value[0])
    L=np.insert(L,0,Ll)
    R=np.append(R,rd.value[1])
    L=np.append(L,Ul)
    pdf_fillD.x=R
    pdf_fillD.y=L
    
    #single point
    y1=np.interp(rd.value[0],Rng,Lx)
    pdf_startD.x=[rd.value[0],rd.value[0]]
    pdf_startD.y=[0,y1]
    y2=np.interp(rd.value[1],Rng,Lx)
    pdf_endD.x=[rd.value[1],rd.value[1]]
    pdf_endD.y=[0,y2]

    


def fillResults():
    thisGame=getGame()
    with ClaimantOut:
        cRes=getClaimantResult(thisGame)
        cResStr=["{:2.2f}".format(f) for f in cRes]
        print(stringifyResult(cResStr))
        ClaimantOut.clear_output(wait=True) 
            
    with DependentOut:
        dRes=getDependentResult(thisGame)
        dResStr=["{:2.2f}".format(f) for f in dRes]
        print (stringifyResult(dResStr))
        DependentOut.clear_output(wait=True) 
    
    updateBQplotC(thisGame,cRes)
    updateBQplotD(thisGame,dRes)


def setFatal():

    if fatal.value:#it's a fatal game
        label.value='Deceased:'
        lab.value='Deceased:'
        ageatdeath.layout.display=None
        h3.layout.display=None
        h5.layout.display=None
        pdf_aadd.display_legend=True
        pdf_aadd.visible=True
        pdf_aad.display_legend=True
        pdf_aad.visible=True

    else:
        label.value='Claimant:'
        lab.value='Claimant:'
        ageatdeath.layout.display='none'
        h3.layout.display='none'
        h5.layout.display='none'
        pdf_aadd.display_legend=False
        pdf_aadd.visible=False
        pdf_aad.display_legend=False
        pdf_aad.visible=False




def on_value_change(change):
    fillResults()


def on_fatal_change(change):
    setFatal()

def on_value_change_age(change):
    if ageatdeath.value>age.value:
        ageatdeath.value=age.value
    ageatdeath.max=age.value
    fillResults()

r.observe(on_value_change,names='value')
fr.observe(on_value_change,names='value')
og.observe(on_value_change,names='value')
op.observe(on_value_change,names='value')        
age.observe(on_value_change_age,names='value')
sex.observe(on_value_change,names='value')
rg.observe(on_value_change,names='value')
ageatdeath.observe(on_value_change,names='value')


rd.observe(on_value_change,names='value')
frd.observe(on_value_change,names='value')
opd.observe(on_value_change,names='value')        
aged.observe(on_value_change,names='value')
sexd.observe(on_value_change,names='value')
rgd.observe(on_value_change,names='value')


deltale.observe(on_value_change,names='value')
deltaled.observe(on_value_change,names='value')

dr.observe(on_value_change, names='value')
og.observe(on_value_change,names='value')
yrattained.observe(on_value_change,names='value')
fatal.observe(on_fatal_change,names='value')

thisGame=getGame()

LxNoI=Lx=Rng=None


In [111]:
curve=thisGame.getClaimant('CLAIMANT').getCurve()
if not r.value[0]==r.value[1]:
    title=curve.getTitle(result=[0,0,0,0],fromAge=r.value[0],toAge=r.value[1],freq=fr.value,cont=1,options=op.value)
else:
    title=curve.getTitle(result=[0,0,0,0],fromAge=r.value[0],freq=fr.value,cont=1,options=op.value)
        
st,en,factor,timeInterval=returnFreq(fr.value)
#The plot
LxNoI,Lx,Rng=curve.getCurve(options=op.value,cont=1)

axes_options={'x':dict(label='Age'),'y':dict(label='Multiplier')}

pdf_figC=plt.figure()
plt.scales(scales={'x':LinearScale(),'y':LinearScale(allow_padding=False)})
pdf_figC.layout.height='400px'
pdf_lineC=plt.plot(Rng,Lx, axes_options=axes_options)
pdf_ageC=plt.vline(age.value,colors=['green'],labels=['Age at trial'], display_legend=True)
pdf_aad=plt.vline(ageatdeath.value,colors=['black'],labels=['Age at death'],display_legend=fatal.value,visible=fatal.value)
mask=(Rng>=r.value[0]) & (Rng<=r.value[1])#Boolean mask
R=Rng[mask]
L=Lx[mask]

pdf_fillC=plt.plot(R,L,fill='bottom',preserve_domain={'x':True,'y':True},colors=['red'],fill_opacities=[0.5])

#single point
y1=np.interp(r.value[0],Rng,Lx)
pdf_startC=plt.plot(colors=['red'] ,preserve_domain={'x': True, 'y': False}, x=[r.value[0], r.value[0]], y=[0, y1])
y2=np.interp(r.value[1],Rng,Lx)
pdf_endC=plt.plot(colors=['red'] ,preserve_domain={'x': True, 'y': False}, x=[r.value[1], r.value[1]], y=[0, y2])



display(pdf_figC)


Figure(axes=[Axis(label='Age', scale=LinearScale()), Axis(label='Multiplier', orientation='vertical', scale=Li…

In [109]:
curve=thisGame.getClaimant('DEPENDENT').getCurve()
if not rd.value[0]==rd.value[1]:
    title=curve.getTitle(result=[0,0,0,0],fromAge=rd.value[0],toAge=rd.value[1],freq=frd.value,cont=1,options=opd.value)
else:
    title=curve.getTitle(result=[0,0,0,0],fromAge=rd.value[0],freq=frd.value,cont=1,options=opd.value)
        
st,en,factor,timeInterval=returnFreq(frd.value)
#The plot
LxNoI,Lx,Rng=curve.getCurve(options=opd.value,cont=1)
pdf_figD=plt.figure()
plt.scales(scales={'x':LinearScale(),'y':LinearScale(allow_padding=False)})
pdf_figD.layout.height='400px'
pdf_lineD=plt.plot(Rng,Lx, axes_options=axes_options)
pdf_ageD=plt.vline(aged.value,colors=['green'],labels=['Age at trial'],display_legend=True)
gap=aged.value-age.value
pdf_aadd=plt.vline(ageatdeath.value+gap, colors=['black'],labels=['Age at death of deceased'],display_legend=fatal.value,visible=fatal.value)

mask=(Rng>=rd.value[0]) & (Rng<=rd.value[1])#Boolean mask
R=Rng[mask]
L=Lx[mask]

pdf_fillD=plt.plot(R,L,fill='bottom',preserve_domain={'x':True,'y':True},colors=['red'],fill_opacities=[0.5])

#single point
y1=np.interp(rd.value[0],Rng,Lx)
pdf_startD=plt.plot([rd.value[0],rd.value[0]],[0,y1],preserve_domain={'x':False,'y':True},colors=['red'])
y2=np.interp(rd.value[1],Rng,Lx)
pdf_endD=plt.plot(colors=['red'] ,preserve_domain={'x': True, 'y': False}, x=[rd.value[1], rd.value[1]], y=[0, y2])




display(pdf_figD)


Figure(axes=[Axis(label='Age', scale=LinearScale()), Axis(label='Multiplier', orientation='vertical', scale=Li…

In [44]:
on_value_change(None)
setFatal()

